In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage
from pathlib import Path
from scipy.io import loadmat
import pickle
from copy import deepcopy

In [ ]:
from bayesee.perception import *
from bayesee.validation import *
from bayesee.visualization import *
from bayesee.generation import *
from bayesee.evaluation import *
from bayesee.iteration import *
from bayesee.partition import *

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
subject = "AE"
file_name = repo_path / f"data/detection/phase-similarity/data_{subject}.pickle"

with open(file_name, "rb") as f:
    stimulus, response = pickle.load(f)

metadata = stimulus["metadata"]
print(metadata.keys())

In [ ]:
stimulus_binned = deepcopy(stimulus)
stimulus_binned["df"] = recursive_bin(stimulus["df"], ["sa", "ss"], [2, 5])
print(stimulus_binned["df"].columns)

In [ ]:
df = stimulus_binned["df"]
df["dprime"] = df["criterion"] = df["PC_max"] = None

unique_bins = df["level2_bin_ss"].unique()
bin_fit_parameters = np.zeros((len(unique_bins), 3))

for index_bin in unique_bins:
    array_index_bin = df["level2_bin_ss"] == index_bin
    array_dprime, array_criterion = compute_dprime_criterion_w_discrete_response(
        {"df": df[array_index_bin]}, {"df": response["df"][array_index_bin]}
    )
    array_PC_max = compute_PC_max_w_dprime(array_dprime)

    df.loc[array_index_bin, "dprime"] = array_dprime
    df.loc[array_index_bin, "criterion"] = array_criterion
    df.loc[array_index_bin, "PC_max"] = array_PC_max

    slope, y_intercept = np.polyfit(
        df.loc[array_index_bin, "amplitude"],
        array_PC_max,
        1,
    )

    x0 = [(norm.cdf(1) - y_intercept) / slope, 2, 0]

    results = minimize(
        negative_loglikelihood_w_parameter,
        x0,
        args=(
            {"df": df[array_index_bin]},
            {"df": response["df"][array_index_bin]},
            unit_likelihood_abc,
        ),
        bounds=((x0[0] / 1.5, x0[0] * 1.5), (x0[1] / 3, x0[1] * 2), (0, 0.5)),
        method="Nelder-Mead",
        options={"maxiter": 500},
    )

    print(results)
    bin_fit_parameters[index_bin - 1, :] = results.x

In [ ]:
n_col = len(unique_bins) // 2

fig, axes = plt.subplots(nrows=2, ncols=n_col, tight_layout=True, figsize=(20, 10))

for index_bin in unique_bins:
    ax = axes[(index_bin - 1) // n_col, (index_bin - 1) % n_col]
    ax2 = ax.twinx()

    array_index_bin = df["level2_bin_ss"] == index_bin
    amplitude_unique, index_unique = np.unique(
        df.loc[array_index_bin, "amplitude"].values, return_index=True
    )

    PC_max_unique = df.loc[array_index_bin, "PC_max"].values[index_unique]
    criterion_unique = df.loc[array_index_bin, "criterion"].values[index_unique]

    ax.scatter(
        amplitude_unique,
        PC_max_unique,
        s=100,
    )

    ax2.scatter(
        amplitude_unique,
        criterion_unique,
        s=100,
        lw=5,
        edgecolors="k",
        facecolors="none",
    )

    amplitude_full = np.linspace(0, amplitude_unique.max() * 1.1, 50)
    a, b, _ = bin_fit_parameters[index_bin - 1, :]

    ax.plot(
        amplitude_full,
        norm.cdf(0.5 * (amplitude_full / a) ** b),
        "-",
        lw=5,
        alpha=0.35,
    )

    ax.set(xlim=[0, 16], ylim=[0.1, 1.1])
    ax2.set(ylim=[-2, 10])

fig.text(
    0.5,
    0,
    "Target amplitude",
    ha="center",
)

fig.text(
    0,
    0.5,
    "PC$_max$",
    va="center",
    rotation=90,
)

fig.text(
    1.01,
    0.45,
    "Criterion",
    va="center",
    rotation=-90,
)

In [ ]:
fig, ax = plt.subplots()

ss_bin_center = np.quantile(
    df.loc[df["level1_bin_sa"] == 1, "ss"], [0.1, 0.3, 0.5, 0.7, 0.9]
)

ax.scatter(
    ss_bin_center,
    bin_fit_parameters[: len(ss_bin_center), 0],
    s=320,
    c="k",
    label="Low $S_A$",
)

ss_bin_center = np.quantile(
    df.loc[df["level1_bin_sa"] == 2, "ss"], [0.1, 0.3, 0.5, 0.7, 0.9]
)

ax.scatter(
    ss_bin_center,
    bin_fit_parameters[len(ss_bin_center) :, 0],
    marker="o",
    s=280,
    facecolors="w",
    linewidths=5,
    edgecolors="k",
    label="High $S_A$",
)